import libraries

In [494]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import pickle as pk 

In [495]:
data = pd.read_csv("data/heart_disease.csv")

drop education columns nad non value/missing rows

In [496]:
data.drop(columns=['education'],inplace=True)

In [497]:
data.dropna(inplace=True)

convert category columns to binary

In [498]:
data['Gender'] = data['Gender'].replace(['Male', 'Female'], [1,0])

C:\Users\adegb\AppData\Local\Temp\ipykernel_13252\410831775.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Gender'] = data['Gender'].replace(['Male', 'Female'], [1,0])


In [499]:
data['Heart_ stroke'] = data['Heart_ stroke'].replace(['yes', 'No'], [1,0])

C:\Users\adegb\AppData\Local\Temp\ipykernel_13252\2440881696.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Heart_ stroke'] = data['Heart_ stroke'].replace(['yes', 'No'], [1,0])


In [500]:
data.prevalentStroke.unique()

array(['no', 'yes'], dtype=object)

In [501]:
data['prevalentStroke'] = data['prevalentStroke'].replace(['yes', 'no'], [1,0])

C:\Users\adegb\AppData\Local\Temp\ipykernel_13252\3553874486.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['prevalentStroke'] = data['prevalentStroke'].replace(['yes', 'no'], [1,0])


partition data into input and output data

In [502]:
inputData = data.drop(columns=['Heart_ stroke'])
outputData = data['Heart_ stroke']

check data distribution to confirm if its balance

In [503]:
outputData.value_counts()

Heart_ stroke
0    3177
1     572
Name: count, dtype: int64

The above result indicate inbalance in the dataset. YES has 572 while NO has 3177. The solution is to perform dataset resampling

In [504]:
sm = SMOTE(random_state=2) #random_state is two because of the array is two(1 &0)

In [505]:
inputData,outputData = sm.fit_resample(inputData, outputData.ravel())

C:\Users\adegb\AppData\Local\Temp\ipykernel_13252\669841114.py:1: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  inputData,outputData = sm.fit_resample(inputData, outputData.ravel())


In [506]:
np.unique(outputData, return_counts=True)

(array([0, 1], dtype=int64), array([3177, 3177], dtype=int64))

Split data into train and test data

In [507]:
x_train, x_test, y_train, y_test = train_test_split(inputData, outputData, test_size=0.30)

Hyperparameter tuning using GridSearchCV

In [508]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}  # Try different values of C
clf = GridSearchCV(LogisticRegression(), param_grid, cv=5)
clf.fit(x_train, y_train)


c:\Users\adegb\miniconda3\envs\Tensor_Env\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\adegb\miniconda3\envs\Tensor_Env\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]})

In [509]:
# Get the best hyperparameters
best_model = clf.best_params_['C']


Train the model with the best hyperparameters

In [510]:
model = LogisticRegression(C=best_model)

fit model

In [511]:
model.fit(x_train, y_train)

c:\Users\adegb\miniconda3\envs\Tensor_Env\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100)

test the model accuracy

In [512]:
ypred_train = model.predict(x_train)
ypred_test = model.predict(x_test)
train_accuracy = accuracy_score(y_train, ypred_train)
test_accuracy = accuracy_score(y_test, ypred_test)

print(f"Best C value: {best_model}")
print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

Best C value: 100
Training accuracy: 0.6427
Test accuracy: 0.6377


test the model with external data 

In [513]:
input = np.array([[0,59,0,0,0,0,1,0,209,150,85,20.77,90,88]])

In [514]:
x = model.predict(input)

c:\Users\adegb\miniconda3\envs\Tensor_Env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [515]:
x[0]

1

export the model using pickle

In [516]:
pk.dump(model, open('model/heart_disease_model.pkl', 'wb'))